# Downscaling Demo


In [ ]:
import os
import sys

import xarray as xr

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")

sk.set_file_destination("downscale_example")

sk.login("SALIENT_USERNAME", "SALIENT_PASSWORD")

## Set geographic bounds

The Salient SDK uses a "Location" object to specify the geographic bounds of a request.


In [ ]:
if False:  # lat/lon
    loc = sk.Location(37.7749, -122.4194)
elif False:  # "shapefile"
    shp_file = sk.upload_bounding_box(-109.1, -102.0, 37.8, 41.0, "Colorado")
    loc = sk.Location(shapefile=shp_file)
elif True:  # "location_file"
    loc_file = sk.upload_location_file(
        lats=[37.7749, 33.9416, 32.7336],
        lons=[-122.4194, -118.4085, -117.1897],
        names=["SFO", "LAX", "SAN"],
        geoname="CA_Airports",
    )
    loc = sk.Location(location_file=loc_file)
else:
    raise ValueError("Invalid location type")

print(loc)

(37.7749, -122.4194)


## Downscale

The `downscale` API endpoint converts Salient's temporally granular weekly/monthly/quarterly forecasts into a daily timeseries ensemble.


In [ ]:
vars = "temp,precip"
start_date = "2021-01-01"
debias = False

file_dsc = sk.downscale(
    loc=loc,
    variables=vars,
    date=start_date,
    members=11,
    debias=debias,
    verbose=False,
    force=False,
)

dsc = xr.load_dataset(file_dsc)
# Later on we'll be merging with data_history, which has a time coordinate:
dsc = dsc.rename({"forecast_day": "time"})

print(dsc)

<xarray.Dataset> Size: 103kB
Dimensions:        (time: 366, location: 1, ensemble: 11)
Coordinates:
  * time           (time) datetime64[ns] 3kB 2021-01-01 ... 2022-01-01
    analog         (ensemble, time) datetime64[ns] 32kB 2021-01-01 ... 1982-1...
    lat            (location) float64 8B 37.77
    lon            (location) float64 8B -122.4
    forecast_date  datetime64[ns] 8B 2021-01-01
Dimensions without coordinates: location, ensemble
Data variables:
    precip_clim    (time, location) float32 1kB 4.549 4.276 ... 5.884 4.549
    temp_clim      (time, location) float32 1kB 9.539 9.476 9.243 ... 9.8 9.539
    precip_anom    (ensemble, time, location) float32 16kB -3.9 -1.504 ... 7.514
    temp_anom      (ensemble, time, location) float32 16kB -0.3568 ... 0.9146
    precip         (ensemble, time, location) float32 16kB 0.6491 ... 12.06
    temp           (ensemble, time, location) float32 16kB 9.182 10.54 ... 10.45


## Historical


In [ ]:
# Match the start and end dates of the downscale timeseries
end_date = dsc.time.max().dt.strftime("%Y-%m-%d").values

file_hst = sk.data_timeseries(
    loc=loc,
    variable=vars,
    field="all",
    start=start_date,
    end=end_date,
    debias=debias,
    verbose=False,
    force=False,
)
# Each variable will download to its own file
print(file_hst)

# Load all of the history files into a single dataset:
hst = sk.load_multihistory(file_hst, fields=["vals"])
print(hst)

                                           file_name variable
0  downscale_example/data_timeseries_fc636fcbaf40...     temp
1  downscale_example/data_timeseries_981833b8b2e7...   precip
<xarray.Dataset> Size: 9kB
Dimensions:  (time: 366, location: 1)
Coordinates:
  * time     (time) datetime64[ns] 3kB 2021-01-01 2021-01-02 ... 2022-01-01
    lat      (location) float64 8B 37.77
    lon      (location) float64 8B -122.4
Dimensions without coordinates: location
Data variables:
    temp     (time, location) float64 3kB 9.404 10.78 10.96 ... 8.064 7.243
    precip   (time, location) float64 3kB 0.000208 1.486 0.2287 ... 0.002722 0.0


# Analyze Downscale vs History

Compare the downscale and history datasets to see how well they match. This is a simple example, but you can use any of the xarray functions to analyze the data.


In [ ]:
dif = dsc - hst

rmse = (dif**2).mean(dim=["ensemble"]) ** 0.5
print(rmse)

<xarray.Dataset> Size: 9kB
Dimensions:        (time: 366, location: 1)
Coordinates:
  * time           (time) datetime64[ns] 3kB 2021-01-01 ... 2022-01-01
    lat            (location) float64 8B 37.77
    lon            (location) float64 8B -122.4
    forecast_date  datetime64[ns] 8B 2021-01-01
Dimensions without coordinates: location
Data variables:
    precip         (time, location) float64 3kB 0.6489 1.216 ... 0.9117 10.88
    temp           (time, location) float64 3kB 0.1796 0.3127 ... 1.16 2.494
